In [1]:
import mysql.connector
import pymongo
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import plotly.express as px
from neo4j import GraphDatabase
import pandas as pd
import json
import dash_bootstrap_components as dbc

In [2]:
########## IMPORTANT ##############

# REPLACE THE USERNAME AND PASSWORD WITH YOUR CREDENTIALS #

def mysql_setup():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="password",
        database="academicworld"
    )
    
    return mydb

In [3]:
########## IMPORTANT ##############

# REPLACE THE URL, URI, USERNAME AND PASSWORD WITH YOUR CREDENTIALS #

client_mongo = pymongo.MongoClient("mongodb://localhost:27017/")
db_mongo = client_mongo["academicworld"]

URI = "neo4j://localhost:7687/academicworld"
AUTH = ("neo4j", "password")

In [4]:
mydb = mysql_setup()
mycursor = mydb.cursor()

# create views

query = "CREATE OR REPLACE VIEW fac_uni as select faculty.id as fac, university.name as uni from faculty inner join university on faculty.university_id = university.id"
mycursor.execute(query)

query = "CREATE OR REPLACE VIEW pub_key as select publication_id, name from publication_keyword inner join keyword on keyword_id = keyword.id"
mycursor.execute(query)

# create procedures

procedure_query0 = "CREATE PROCEDURE IF NOT EXISTS best_pub_key(IN keyword varchar(255), IN criteria varchar(255), IN lim int) BEGIN set @k = keyword; set @query = CONCAT ('select id, title, year, num_citations, venue, X.score from publication inner join (select publication_id, score from publication_keyword, keyword where publication_keyword.keyword_id = keyword.id and keyword.name = (?)) X on publication.id = X.publication_id order by ', criteria,' desc limit ', lim); PREPARE stmt FROM @query; EXECUTE stmt using @k; DEALLOCATE PREPARE stmt; END"
mycursor = mydb.cursor()
mycursor.execute(procedure_query0)

procedure_query1 = "CREATE PROCEDURE IF NOT EXISTS top_rel_keys(IN keyword varchar(255), IN lim int) BEGIN set @k = keyword; set @query = CONCAT ('select name, total from keyword inner join (select keyword_id, count(keyword_id) as total from publication_keyword where publication_id in (select publication_id from publication_keyword where keyword_id in (select id from keyword where name = (?))) group by keyword_id) X on keyword.id = X.keyword_id where name <> (?) order by total desc limit ', lim); PREPARE stmt FROM @query; EXECUTE stmt using @k, @k; DEALLOCATE PREPARE stmt; END"
mycursor = mydb.cursor()
mycursor.execute(procedure_query1)


In [5]:
def get_best_publications_by_keyword(keyword, criteria = "keyword score", limit = 10):
    mydb = mysql_setup()
    if limit == 'All':
        limit = 300000
    # criteria can be either score or num_citations
    if criteria == "keyword score":
        c = "score"
    else:
        c = "num_citations"
    #query = "select id, title, year, num_citations, venue, X.score from publication inner join (select publication_id, score from publication_keyword, keyword where publication_keyword.keyword_id = keyword.id and keyword.name = %s) X on publication.id = X.publication_id order by " + criteria + " desc limit %s" 
    mycursor = mydb.cursor()
    mycursor.callproc('best_pub_key', (keyword, c, limit))
    for res in mycursor.stored_results():
        myresult = res.fetchall()
        df = pd.DataFrame(myresult, columns=['Id', 'Title', 'Year', '# of citations', 'Venue', 'Score'])
        mycursor.close()
        return df
        

In [6]:
#get_best_publications_by_keyword("data mining", criteria = "num_citations")

In [7]:
#get_best_publications_by_keyword("data mining", criteria = "keyword score")

In [8]:
def get_best_faculty_by_keyword(keyword, criteria = "keyword score", limit = 10):
    if criteria == "keyword score":
        col_name = "Keyword Score"
        query = "select faculty.id, faculty.name, faculty.position, faculty.research_interest, faculty.email, faculty.phone, faculty.photo_url, university.name, score from (faculty inner join university on faculty.university_id = university.id) inner join (select faculty_id, score from faculty_keyword where keyword_id in (select id from keyword where name = %s)) X on faculty.id=X.faculty_id order by X.score desc limit %s"
    elif criteria == "publication score":
        col_name = "Total Publication Score"
        query = "select faculty.id, faculty.name, faculty.position, faculty.research_interest, faculty.email, faculty.phone, faculty.photo_url, university.name, total from (faculty inner join university on faculty.university_id = university.id) inner join (select faculty_id, sum(score) as total from (faculty_publication inner join publication_keyword on faculty_publication.publication_id = publication_keyword.publication_id) inner join keyword on publication_keyword.keyword_id = keyword.id and keyword.name = %s group by faculty_id) X on faculty.id = X.faculty_id order by X.total desc limit %s" 
    elif criteria == "no. of citations":
        col_name = "Total # of Citations"
        query = "select faculty.id, faculty.name, faculty.position, faculty.research_interest, faculty.email, faculty.phone, faculty.photo_url, university.name, total from (faculty inner join university on faculty.university_id = university.id) inner join (select faculty_id, sum(num_citations) as total from ((faculty_publication inner join publication on faculty_publication.publication_id = publication.id) inner join publication_keyword on faculty_publication.publication_id = publication_keyword.publication_id) inner join keyword on publication_keyword.keyword_id = keyword.id and keyword.name = %s group by faculty_id) X on faculty.id = X.faculty_id order by X.total desc limit %s"
    else:
        return "invalid criteria"
    mycursor = mydb.cursor(prepared=True)
    mycursor.execute(query, (keyword, str(limit)))
    myresult = mycursor.fetchall()
    
    df = pd.DataFrame(myresult, columns=['Id', 'Name', 'Position', 'Research Interest', 'Email', 'Phone', 'Photo', 'University', col_name])
    return df
    

In [9]:
#get_best_faculty_by_keyword("data mining", criteria = "keyword_score")

In [10]:
#get_best_faculty_by_keyword("data mining", criteria = "publication_score")

In [11]:
#get_best_faculty_by_keyword("data mining", criteria = "num_citations")

In [12]:
def get_best_university_by_keyword(keyword, criteria = "keyword score", limit = 10):
    mydb = mysql_setup()
    if limit == 'All':
        limit = 300000
    if criteria == "keyword score":
        col_name = "Total Keyword Score"
        query = "select uni, sum(X.score) from fac_uni inner join (select faculty_id, score from faculty_keyword where keyword_id in (select id from keyword where name = %s)) X on fac=X.faculty_id group by uni order by sum(X.score) desc limit %s"
    elif criteria == "publication score":
        col_name = "Total Publication Score"
        query = "select uni, sum(X.total) from fac_uni inner join (select faculty_id, sum(score) as total from (faculty_publication inner join publication_keyword on faculty_publication.publication_id = publication_keyword.publication_id) inner join keyword on publication_keyword.keyword_id = keyword.id and keyword.name = %s group by faculty_id) X on fac = X.faculty_id group by uni order by sum(X.total) desc limit %s"
    elif criteria == "no. of citations":
        col_name = "Total # of Citations"
        query = "select uni, sum(X.total) from fac_uni inner join (select faculty_id, sum(num_citations) as total from ((faculty_publication inner join publication on faculty_publication.publication_id = publication.id) inner join publication_keyword on faculty_publication.publication_id = publication_keyword.publication_id) inner join keyword on publication_keyword.keyword_id = keyword.id and keyword.name = %s group by faculty_id) X on fac = X.faculty_id group by uni order by sum(X.total) desc limit %s" 
    else:
        return "invalid criteria"
    mycursor = mydb.cursor(prepared=True)
    mycursor.execute(query, (keyword, str(limit)))
    myresult = mycursor.fetchall()
    
    df = pd.DataFrame(myresult, columns=['University', col_name])
    mycursor.close()
    return df

In [13]:
#get_best_university_by_keyword("data mining", criteria = "keyword score")

In [14]:
#get_best_university_by_keyword("data mining", criteria = "publication score")

In [15]:
#get_best_university_by_keyword("data mining", criteria = "no. of citations")

In [16]:
def get_top_related_keywords(keyword, limit = 10):
    #query = "select name, total from keyword inner join (select keyword_id, count(keyword_id) as total from publication_keyword where publication_id in (select publication_id from publication_keyword where keyword_id in (select id from keyword where name = %s)) group by keyword_id) X on keyword.id = X.keyword_id where name <> %s order by total desc limit %s"
    mydb = mysql_setup()
    mycursor = mydb.cursor()
    myresult = mycursor.callproc('top_rel_keys', (keyword, limit))
    for res in mycursor.stored_results():
        myresult = res.fetchall()
        df = pd.DataFrame(myresult, columns=['Keyword', 'Total Simultaneous Occurences'])
        return df
    

In [17]:
#get_top_related_keywords("data mining")

In [18]:
def get_popularity_over_time(keyword, start_year = None, end_year = None):
    mydb = mysql_setup()
    mycursor = mydb.cursor(prepared = True)
    if start_year != None and end_year != None:
        query = "select year, count(id) from publication where year between %s and %s and id in (select publication_id from pub_key where name = %s) group by year order by year desc"
        mycursor.execute(query, (start_year, end_year, keyword))
    elif start_year != None:
        query = "select year, count(id) from publication where year >= %s and id in (select publication_id from pub_key where name = %s) group by year order by year desc"
        mycursor.execute(query, (start_year, keyword))
    elif end_year != None:
        query = "select year, count(id) from publication where year <= %s and id in (select publication_id from pub_key where name = %s) group by year order by year desc"
        mycursor.execute(query, (end_year, keyword))
    else:
        query = "select year, count(id) from publication where id in (select publication_id from pub_key where name = %s) group by year order by year desc"
        mycursor.execute(query, (keyword,))
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult, columns=['Year', '# of Publications'])
    return df
    
    

In [19]:
#get_popularity_over_time('science')

In [20]:
def get_matching_publications(title):
    collection = db_mongo["publications"]
    query = {"title": { "$regex": title }}
    documents = collection.find(query)
    myresult = []
    for x in documents:
        myresult.append([x['id'], x['title'], x['venue'], x['year'], x['numCitations']])
    df = pd.DataFrame(myresult, columns=['Id', 'Title', 'Venue', 'Year', '# of Citations'])
    return df



In [21]:
#get_matching_publications("dations of Machine Lea").to_dict('records')

In [22]:
def add_to_saved_list(id):
    try:
        collection = db_mongo["publications"]
        query = {"id": id}
        documents = collection.find(query)
        saved_list = db_mongo["saved_list"]
        for x in documents:
            saved_list.insert_one(x)
    except:
        return "duplicate"


In [23]:
#add_to_saved_list(1944672)

In [24]:
def delete_from_saved_list(id):
    try:
        collection = db_mongo["saved_list"]
        query = {"id": id}
        documents = collection.delete_one(query);
    except:
        return "error"


In [25]:
#delete_from_saved_list(2142827986)

In [26]:
def show_saved_list():
    saved_list = db_mongo["saved_list"]
    documents = saved_list.find()
    myresult = []
    for x in documents:
        myresult.append([x['id'], x['title'], x['venue'], x['year'], x['numCitations']])
    df = pd.DataFrame(myresult, columns=['Id', 'Title', 'Venue', 'Year', '# of Citations'])
    return df
    

In [27]:
#show_saved_list()

In [28]:
new_keyword_count = 0

def add_keywords(publication_id, keyword):
    global new_keyword_count
    with GraphDatabase.driver(URI, auth=AUTH) as driver:

        driver.verify_connectivity()
        
        mydb = mysql_setup()
        mycursor = mydb.cursor()
        query = "select count(*) from publication where id = %s"
        mycursor.execute(query, (publication_id,))
        myresult = mycursor.fetchall()
        df = pd.DataFrame(myresult, columns=['count'])
        publication_exists = df.iloc[0,0] != 0

        records, summary, keys = driver.execute_query(
            "MATCH (p:PUBLICATION {id: '" + publication_id + "'}) WITH COUNT(p) > 0  as pub_exists RETURN pub_exists",
            database_="academicworld",
        )
        pub_exists = records[0]['pub_exists']
        
        if not (publication_exists or pub_exists):
            return "No such publication exists."

        mydb = mysql_setup()
        mycursor = mydb.cursor()
        query = "select count(*) from publication_keyword where publication_id = %s and keyword_id = (select id from keyword where name = %s)"
        mycursor.execute(query, (publication_id, keyword))
        myresult = mycursor.fetchall()
        df = pd.DataFrame(myresult, columns=['count'])
        relationship_exists = df.iloc[0,0] != 0

        records, summary, keys = driver.execute_query(
            "OPTIONAL MATCH  (p:PUBLICATION {id: '" + publication_id + "'}), (k:KEYWORD {name: '" + keyword + "'}) RETURN EXISTS( (p)-[:LABEL_BY]->(k) ) as exs",
            database_="academicworld",
        )
        rel_exists = records[0]['exs']

        if relationship_exists or rel_exists:
            return "Keyword already exists in publication."

        if (pub_exists):
            records, summary, keys = driver.execute_query(
                "OPTIONAL MATCH (k:KEYWORD {name: '" + keyword + "'}) RETURN k IS NOT NULL AS existance",
                database_="academicworld",
            )
            keyword_exists = records[0]
            if keyword_exists["existance"]:
                query = "MATCH (p:PUBLICATION {id: '" + publication_id + "'}), (k:KEYWORD {name: '" + keyword + "'}) CREATE (p)-[:LABEL_BY]->(k)"
            else:
                query = "MATCH (p:PUBLICATION {id: '" + publication_id + "'}) CREATE (p)-[:LABEL_BY]->(k:KEYWORD {id: 'k_" + str(new_keyword_count) + "', name: '" + keyword + "'})"
        
            driver.execute_query(
                query,
                database_="academicworld",
            )
            
        elif publication_exists:
            mydb = mysql_setup()
            mycursor = mydb.cursor()
            
            query = "select id from keyword where name = %s"
            mycursor.execute(query, (keyword,))
            myresult = mycursor.fetchall()
            df = pd.DataFrame(myresult, columns=['id'])
            key_exists = len(df['id']) != 0

            if key_exists:
                id_num = str(df['id'][0])
            else:
                id_num = str(84000+new_keyword_count)
                mydb = mysql_setup()
                mycursor = mydb.cursor()
                query = "insert into keyword (id, name) values (%s, %s)"
                mycursor.execute(query, (id_num, keyword))
                mydb.commit()

            mydb = mysql_setup()
            mycursor = mydb.cursor()
            query = "insert into publication_keyword (publication_id, keyword_id) values (%s, %s)"
            mycursor.execute(query, (publication_id, id_num))
            mydb.commit()
            
    new_keyword_count += 1
    return "Keyword successfully added!"

In [29]:
#add_keywords('21366', 'hardware')

In [30]:
# Initialize the app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App layout
app.layout = html.Div([
    html.Div([
    html.H1('Keyword-based analysis of the Academic World database',),
    html.H4('by Sasandi Hettiarachchi'),],
            className='text-center', style={'backgroundColor':'#0FC2C0', 'paddingTop': '8px', 'paddingBottom': '8px'}),
    dbc.Row(
        [dbc.Col(
            html.Div([
                html.H3('Best publications by keyword', style={'backgroundColor':'#DAD6D7', 'padding': '8px', 'margin': '0px'}),
                dbc.Row([
                    dbc.Col(
                        html.Div([
                            "Keyword: ",
                            dcc.Input(id='pub-keyword-input', value='', type='text'),
                        ]),
                    ),
                    dbc.Col(
                        html.Div([
                            "Criteria: ",
                            dcc.Dropdown(['keyword score', 'no. of citations'], 'keyword score', id='pub-criteria-dropdown', searchable=False),
                        ]),
                    ),
                    dbc.Col(
                        html.Div([
                            "No. of results: ",
                            dcc.Dropdown([10, 20, 50, 100, 'All'], 10, id='pub-no-of-results-dropdown', searchable=False),
                        ]),
                    )
                ], style={'margin': '2px'}),
                html.Div([
                    dash_table.DataTable(id='best-pub-table', data=None, page_size=10, 
                        style_cell={
                            'maxWidth': 0,
                            'overflow': 'hidden',
                            'textOverflow': 'ellipsis',
                    },),
                ]),
            ]),
            style={'outline':'1px solid', 'marginLeft': '12px', 'marginRight': '6px', 'padding': '0px'}
        ),
        dbc.Col(
            html.Div([
                html.H3('Best faculty by keyword', style={'backgroundColor':'#DAD6D7', 'padding': '8px'}),
                dbc.Row([
                    dbc.Col(
                        html.Div([
                            "Keyword: ",
                            dcc.Input(id='fac-keyword-input', value='', type='text'),
                        ]),
                    ),
                    dbc.Col(
                        html.Div([
                            "Criteria: ",
                            dcc.Dropdown(['keyword score', 'publication score', 'no. of citations'], 'keyword score', id='fac-criteria-dropdown', searchable=False),
                        ]),
                    ),
                    dbc.Col(
                        html.Div([
                            "No. of results: ",
                            dcc.Dropdown([10, 20, 50, 100, 'All'], 10, id='fac-no-of-results-dropdown', searchable=False),
                        ]),
                    )
                ], style={'margin': '2px'}),
                html.Div(
                    dbc.Pagination(id="best-fac-pagination", max_value=10), style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginTop': '8px'}
                ),
                dbc.Row([
                    dbc.Col(
                        dbc.CardImg(id="best-fac-photo", src="https://dash-bootstrap-components.opensource.faculty.ai/static/images/placeholder286x180.png", top=True),
                    ),
                # 'Id', 'Name', 'Position', 'Research Interest', 'Email', 'Phone', 'Photo', 'University'
                    dbc.Col(
                        dbc.CardBody([
                            html.H3(id="best-fac-name"),
                            html.P(id="best-fac-id"),
                            html.P(id="best-fac-position"),
                            html.P(id="best-fac-research-interest"),
                            html.P(id="best-fac-email"),
                            html.P(id="best-fac-phone"),
                            html.P(id="best-fac-university"),
                            ]),
                    ),
                ], style={'padding': '4px'}),                   
            ]),
            style={'outline':'1px solid', 'marginRight': '12px', 'marginLeft': '6px', 'padding': '0px'}
        )], style={'marginTop': '8px', 'display': 'flex'}),
    dbc.Row(
        [dbc.Col(
            html.Div([
            html.H3('Best universities by keyword', style={'backgroundColor':'#DAD6D7', 'padding': '8px'}),
            dbc.Row([
                dbc.Col(
                    html.Div([
                        "Keyword: ",
                        dcc.Input(id='uni-keyword-input', value='', type='text'),
                    ]),
                ),
                dbc.Col(
                    html.Div([
                        "Criteria: ",
                        dcc.Dropdown(['keyword score', 'publication score', 'no. of citations'], 'keyword score', id='uni-criteria-dropdown', searchable=False),
                    ]),
                ),
                dbc.Col(
                    html.Div([
                        "No. of results: ",
                        dcc.Dropdown([10, 20, 50, 100, 'All'], 10, id='uni-no-of-results-dropdown', searchable=False),
                    ]),
                )
            ], style={'margin': '2px'}),
            dash_table.DataTable(id='best-uni-table', data=None, page_size=10),
        ]) ,style={'outline':'1px solid', 'marginRight': '6px', 'marginLeft': '12px', 'padding': '0px'}),
         dbc.Col(
             [
             html.H3('Saved publications', style={'backgroundColor':'#DAD6D7', 'padding': '8px'}),
             html.Div(
                [html.Div([
                html.P(id='saved-publications-search-term', children='Search: '),
                dbc.Row([
                    dbc.Col(
                        html.Div([
                            dcc.Dropdown(options=[], id='saved-publications-search-dropdown', multi=True),
                        ]), width = 8
                    ),
                    dbc.Col(
                        html.Button('Add to list', id='saved-publications-add-to-list-button', n_clicks=0), width = 4
                    )
                ]),
                html.Div(
                    dash_table.DataTable(id='saved-publications-list', data=None, page_size=10, row_deletable=True, style_cell={
                            'maxWidth': 0,
                            'overflow': 'hidden',
                            'textOverflow': 'ellipsis',
                    }),
                ),
            ]),
            html.Div([
                html.H4('Add keyword to publication'),
                dbc.Row([
                    dbc.Col(
                        html.Div([
                            "Publication ID: ",
                            dcc.Input(id='add-keyword-publication-id', value='', type='text'),
                        ]),
                    ),
                    dbc.Col(
                        html.Div([
                            "Keyword: ",
                            dcc.Input(id='add-keyword-input', value='', type='text'),
                        ]),
                    ),
                    dbc.Col([
                        html.Br(),
                        html.Button('Add to publication', id='add-keyword-button', n_clicks=0),
                    ]),
                ]),
                dbc.Modal(
                    [
                        dbc.ModalHeader(dbc.ModalTitle("Add keyword to publication")),
                        dbc.ModalBody("Keyword successfully added!", id="add-keyword-modal-body"),
                        dbc.ModalFooter(
                            dbc.Button(
                                "Close", id="add-keyword-close-modal-button", n_clicks=0
                            )
                        ),
                    ],
                    id="add-keyword-modal",
                    is_open=False,
                ),
            ], style={'marginTop': '8px'})]
       ,style = {'padding': '6px'}
    )] ,style={'outline':'1px solid', 'marginRight': '12px', 'marginLeft': '6px', 'padding': '0px'})]
    , style={'marginTop': '12px'}),
    dbc.Row(
        [dbc.Col(
            html.Div([
                html.H3('Top related keywords', style={'backgroundColor':'#DAD6D7', 'padding': '8px'}),
                dbc.Row([
                    dbc.Col(
                        html.Div([
                            "Keyword: ",
                            dcc.Input(id='top-related-keyword-input', value='', type='text'),
                        ]),
                    ),
                    dbc.Col(
                        dbc.Row([
                            dbc.Col("No. of results: ", width = 5),
                            dbc.Col(dcc.Dropdown([10, 20, 50], 10, id='top-related-keyword-no-of-results-dropdown', searchable=False), width = 7),
                        ]),
                    ),
                ], style={'margin': '2px'}),
                dcc.Graph(id="top-related-keyword-graph"),
            ]),
            style={'outline':'1px solid', 'marginRight': '6px', 'marginLeft': '12px', 'padding': '0px'}
        ),
         dbc.Col(
            html.Div([
                html.H3('Popularity over time', style={'backgroundColor':'#DAD6D7', 'padding': '8px'}),
                html.Div([
                html.Div([
                    "Keyword: ",
                    dcc.Input(id='popularity-over-time-keyword-input', value='', type='text'),
                ]),
                html.Div([
                    "Year range",
                    dcc.RangeSlider(min=1960, max=2024, step=None, marks={i: '{}'.format(i) if i%5 == 0 or i == 2024 else '' for i in range(1960,2025)},  value=[1960, 2024], updatemode='drag', id='popularity-over-time-range-slider'),
                ]),
                ], style={'marginLeft': '6px'}),
                dcc.Graph(id="popularity-over-time-graph"),
            ]),
             style={'outline':'1px solid', 'marginRight': '12px', 'marginLeft': '6px', 'padding': '0px'}
         )], style={'marginTop': '12px'}
    ),
    dcc.Store(id='best-fac-data', data=[]),
    dcc.Store(id='publications-search-term', data=''),
    dcc.Store(id='saved-publications-ids', data=[])
], style={'margin': '12px', 'fontFamily': "Gill Sans"})

def convert_records_to_tooltips(records):
    return [{column: {'value': str(value), 'type': 'markdown'} for column, value in row.items()} for row in records]

@callback(
    Output(component_id='best-pub-table', component_property='data'),
    Output(component_id='best-pub-table', component_property='tooltip_data'),
    Input(component_id='pub-keyword-input', component_property='value'),
    Input(component_id='pub-criteria-dropdown', component_property='value'),
    Input(component_id='pub-no-of-results-dropdown', component_property='value')
)
def update_pub_div(keyword, criteria, limit):
    pub_df = get_best_publications_by_keyword(keyword, criteria, limit)
    records = pub_df.to_dict('records')
    return records, convert_records_to_tooltips(records)

@callback(
    Output(component_id='best-fac-data', component_property='data'),
    Input(component_id='fac-keyword-input', component_property='value'),
    Input(component_id='fac-criteria-dropdown', component_property='value'),
    Input(component_id='fac-no-of-results-dropdown', component_property='value')
)
def update_fac_div(keyword, criteria, limit):
    fac_df = get_best_faculty_by_keyword(keyword, criteria, limit)
    return fac_df.to_dict('records')

@callback(
    Output(component_id='best-uni-table', component_property='data'),
    Input(component_id='uni-keyword-input', component_property='value'),
    Input(component_id='uni-criteria-dropdown', component_property='value'),
    Input(component_id='uni-no-of-results-dropdown', component_property='value')
)
def update_uni_div(keyword, criteria, limit):
    uni_df = get_best_university_by_keyword(keyword, criteria, limit)
    return uni_df.to_dict('records')

@callback(
    Output(component_id='top-related-keyword-graph', component_property='figure'),
    Input(component_id='top-related-keyword-input', component_property='value'),
    Input(component_id='top-related-keyword-no-of-results-dropdown', component_property='value'),
)
def update_top_related_keywords(keyword, limit):
    keyword_df = get_top_related_keywords(keyword, limit)
    fig = px.bar(keyword_df, x='Keyword', y='Total Simultaneous Occurences')
    return fig

@callback(
    Output(component_id='add-keyword-modal', component_property='is_open'),
    Output(component_id='add-keyword-modal-body', component_property='children'),
    Input(component_id='add-keyword-button', component_property='n_clicks'),
    State(component_id='add-keyword-publication-id', component_property='value'),
    State(component_id='add-keyword-input', component_property='value'),
    prevent_initial_call=True
)
def add_keyword_to_publication(btn, id, keyword):
    return True, add_keywords(id, keyword)

@callback(
    Output("add-keyword-modal", "is_open", allow_duplicate=True),
    Input("add-keyword-close-modal-button", "n_clicks"),
    prevent_initial_call=True
)
def close_modal(n1):
    return False

@callback(
    Output(component_id='popularity-over-time-graph', component_property='figure'),
    Input(component_id='popularity-over-time-keyword-input', component_property='value'),
    Input(component_id='popularity-over-time-range-slider', component_property='value'),
)
def update_popularity_over_time(keyword, years):
    popularity_df = get_popularity_over_time(keyword, years[0], years[1]).sort_values(by=['Year'])
    fig = px.line(popularity_df, x='Year', y='# of Publications')
    return fig

@callback(
    Output(component_id='best-fac-id', component_property='children'),
    Output(component_id='best-fac-position', component_property='children'),
    Output(component_id='best-fac-research-interest', component_property='children'),
    Output(component_id='best-fac-email', component_property='children'),
    Output(component_id='best-fac-phone', component_property='children'),
    Output(component_id='best-fac-university', component_property='children'),
    Output(component_id='best-fac-name', component_property='children'),
    Output(component_id='best-fac-photo', component_property='src'),
    Input(component_id='best-fac-data', component_property='data'),
    Input(component_id='best-fac-pagination', component_property='active_page'),
)
def display_best_fac_profile(data, active_page):
    rows = ['Id', 'Position', 'Research Interest', 'Email', 'Phone', 'University']
    index = active_page - 1 if active_page is not None else 0
    if len(data) <= index:
        return ['' for row_name in rows] + ['' , 'https://dash-bootstrap-components.opensource.faculty.ai/static/images/placeholder286x180.png']
    fac_data = data[index]
    return [row_name + ': ' + str(fac_data[row_name]) for row_name in rows] + [fac_data['Name'], fac_data['Photo']]

@callback(
    Output(component_id='publications-search-term', component_property='data'),
    Output(component_id='saved-publications-search-term', component_property='children'),
    Input(component_id='publications-search-term', component_property='data'),
    Input(component_id='saved-publications-search-dropdown', component_property='search_value'),
)
def set_publication_search_keyword(cur_keyword, new_keyword):
    if new_keyword == None or new_keyword == '':
        return cur_keyword, 'Search: ' + cur_keyword
    return new_keyword, 'Search: ' + new_keyword

@callback(
    Output(component_id='saved-publications-list', component_property='data'),
    Output(component_id='saved-publications-list', component_property='tooltip_data'),
    Output(component_id='saved-publications-ids', component_property='data'),
    Output(component_id='saved-publications-search-dropdown', component_property='value'),
    Input(component_id='saved-publications-add-to-list-button', component_property='n_clicks'),
    State(component_id='saved-publications-search-dropdown', component_property='value'),
)
def add_publications_to_list(n_clicks, ids):
    if ids == None:
        records = show_saved_list().to_dict('records')
        return records, convert_records_to_tooltips(records), [record["Id"] for record in records], ids
    for id in ids:
        add_to_saved_list(id)
    records = show_saved_list().to_dict('records')
    return records, convert_records_to_tooltips(records), [record["Id"] for record in records], []

@callback(
    Output(component_id='saved-publications-ids', component_property='data', allow_duplicate=True),
    Output(component_id='saved-publications-list', component_property='tooltip_data', allow_duplicate=True),
    Input(component_id='saved-publications-list', component_property='data'),
    Input(component_id='saved-publications-ids', component_property='data'),
    prevent_initial_call=True
)
def delete_publications_from_list(cur_records, prev_ids):
    cur_ids = [record["Id"] for record in cur_records]
    for id in prev_ids:
        if id not in cur_ids:
            delete_from_saved_list(id)
    records = show_saved_list().to_dict('records')
    return cur_ids, convert_records_to_tooltips(records)

@callback(
    Output(component_id='saved-publications-search-dropdown', component_property='options'),
    Input(component_id='publications-search-term', component_property='data'),
)
def display_publication_search_results(keyword):
    if keyword == None or keyword == '':
        return []
    publications = get_matching_publications(keyword).sort_values(by=['# of Citations'], ascending=False).to_dict('records')
    # Title	Venue	Year	# of Citations
    return [{"label": pub['Title'] + ' (' + str(pub['Year']) + ') - ' + str(pub['# of Citations']) + ' citations', "value": pub["Id"]} for pub in publications[:10]]
    
# Run the app
if __name__ == '__main__':
    app.run(debug=True)